In [57]:
import numpy as np 

import pandas as pd

import os
from pathlib import Path

from openmm.unit import Quantity, Unit
from openmm.unit import kelvin, bar, litre, kilojoule_per_mole, mole, nanometer, angstrom, kilocalorie_per_mole
from openmm.unit import AVOGADRO_CONSTANT_NA, BOLTZMANN_CONSTANT_kB
R = BOLTZMANN_CONSTANT_kB * AVOGADRO_CONSTANT_NA # gas constant

## OPC to GOPAL

In [58]:
df = pd.read_csv('Force_field_parameters.csv')
df.iloc[[0,9,13],:]

,Water,OH,HH,HOH,OM,Charge O,Charge H,c6,c12,Sigma,...,Unnamed: 11,Unnamed: 12,Unnamed: 13,sinthetaby2,costhetaby2,theta from costhetaby2,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20
0,FF,nm,nm,Deg,nm,e,e,kJ/mol-nm3,kJ/mol-nm6,nm,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,OPC,0.0872400000,0.13711631100,103.60000,0.015940000,-1.3582000000,0.6791000000,0.0035905056,3.620052370E-06,0.31666,...,0.14772952080,0.14772952080,NaN,0.79,0.62,103.6000,0.00E+00,NaN,NaN,NaN
13,GOPAL,0.0894642608,0.14166163700,104.69344,0.015192861,-1.2676837700,0.6338418850,0.0032850341,3.288916150E-06,0.31629,...,0.13899619534,0.13899619534,NaN,0.79,0.61,104.6934,0.00E+00,NaN,NaN,NaN


In [59]:
red_df=df.iloc[[9,13],:].copy()

In [69]:
def conversion(column, current_unit, wanted_unit):
    series=red_df[column]
    lst=series.to_list()
    values=[float(x) for x in lst]
    values_init=values*current_unit
    values_final=values_init.in_units_of(wanted_unit)
    print('Initial values:',values_init)
    print('Final values', values_final)
    results={}
    results['OPC_initial']=values_init[0]
    results['GOPAL_initial']=values_init[1]
    results['OPC_final']=values_final[0]
    results['GOPAL_final']=values_final[1]

    return results

In [70]:
conversion('OH',nanometer,angstrom)

Initial values: [0.08724, 0.0894642608] nm
Final values [0.8724, 0.8946426080000001] A


{'OPC_initial': Quantity(value=0.08724, unit=nanometer),
 'GOPAL_initial': Quantity(value=0.0894642608, unit=nanometer),
 'OPC_final': Quantity(value=0.8724, unit=angstrom),
 'GOPAL_final': Quantity(value=0.8946426080000001, unit=angstrom)}

In [71]:
conversion('HH', nanometer,angstrom)

Initial values: [0.137116311, 0.141661637] nm
Final values [1.37116311, 1.41661637] A


{'OPC_initial': Quantity(value=0.137116311, unit=nanometer),
 'GOPAL_initial': Quantity(value=0.141661637, unit=nanometer),
 'OPC_final': Quantity(value=1.37116311, unit=angstrom),
 'GOPAL_final': Quantity(value=1.41661637, unit=angstrom)}

In [72]:
conversion('OM', nanometer,angstrom)

Initial values: [0.01594, 0.015192861] nm
Final values [0.1594, 0.15192861] A


{'OPC_initial': Quantity(value=0.01594, unit=nanometer),
 'GOPAL_initial': Quantity(value=0.015192861, unit=nanometer),
 'OPC_final': Quantity(value=0.1594, unit=angstrom),
 'GOPAL_final': Quantity(value=0.15192861, unit=angstrom)}

In [73]:
conversion('c6', (kilojoule_per_mole/nanometer**3),(kilocalorie_per_mole/angstrom**3))

Initial values: [0.0035905056, 0.0032850341] kJ/(nm**3 mol)
Final values [8.581514340344169e-07, 7.851419933078394e-07] kcal/(A**3 mol)


{'OPC_initial': Quantity(value=0.0035905056, unit=kilojoule/(nanometer**3*mole)),
 'GOPAL_initial': Quantity(value=0.0032850341, unit=kilojoule/(nanometer**3*mole)),
 'OPC_final': Quantity(value=8.581514340344169e-07, unit=kilocalorie/(angstrom**3*mole)),
 'GOPAL_final': Quantity(value=7.851419933078394e-07, unit=kilocalorie/(angstrom**3*mole))}

In [74]:
conversion('c12', (kilojoule_per_mole/nanometer**6),(kilocalorie_per_mole/angstrom**6))

Initial values: [3.62005237e-06, 3.28891615e-06] kJ/(nm**6 mol)
Final values [8.652132815487571e-13, 7.860698255258125e-13] kcal/(A**6 mol)


{'OPC_initial': Quantity(value=3.62005237e-06, unit=kilojoule/(nanometer**6*mole)),
 'GOPAL_initial': Quantity(value=3.28891615e-06, unit=kilojoule/(nanometer**6*mole)),
 'OPC_final': Quantity(value=8.652132815487571e-13, unit=kilocalorie/(angstrom**6*mole)),
 'GOPAL_final': Quantity(value=7.860698255258125e-13, unit=kilocalorie/(angstrom**6*mole))}

In [76]:
sigma=conversion('Sigma ', nanometer,angstrom)

Initial values: [0.31666, 0.31629] nm
Final values [3.1666, 3.1629] A


In [95]:
def sigma2rmin(sigma:Quantity)-> Quantity:
    #rmin2=rmin2.in_units_of(angstrom)
    rmin=sigma*(2**(1/6))
    rmin_half=rmin/2
    print('Sigma=',sigma)
    print('Rmin=',rmin)
    print('Rmin_half=',rmin_half)
    return rmin, rmin_half


In [96]:
rmin_opc=sigma2rmin(sigma['OPC_final'])

Sigma= 3.1666 A
Rmin= 3.5543883221764605 A
Rmin_half= 1.7771941610882303 A


In [97]:
rmin_gopal=sigma2rmin(sigma['GOPAL_final'])

Sigma= 3.1629 A
Rmin= 3.550235212597716 A
Rmin_half= 1.775117606298858 A


In [98]:
rmin_gopal[1]

Quantity(value=1.775117606298858, unit=angstrom)

In [99]:
conversion('Epsilon', kilojoule_per_mole, kilocalorie_per_mole)

Initial values: [0.8903, 0.82029] kJ/mol
Final values [0.21278680688336518, 0.1960540152963671] kcal/mol


{'OPC_initial': Quantity(value=0.8903, unit=kilojoule/mole),
 'GOPAL_initial': Quantity(value=0.82029, unit=kilojoule/mole),
 'OPC_final': Quantity(value=0.21278680688336518, unit=kilocalorie/mole),
 'GOPAL_final': Quantity(value=0.1960540152963671, unit=kilocalorie/mole)}

In [ ]:
from openff.toolkit.typing.engines.smirnoff import forcefield, ForceField
from openff.evaluator.forcefield import SmirnoffForceFieldSource

# forcefield.get_available_force_fields()

In [ ]:
ff_path = ForceField("opc.offxml")

In [ ]:
# from openff.toolkit import Topology

# gopal_top = Topology.__init__(['GOPAL.top'])

In [ ]:
ff_path.reg